Добавлен бутстрап-анализ для расчета доверительных интервалов:
- Для precision и recall используется бутстрап по классам
- Для mAP используется нормальное приближение (так как это скалярная метрика)

Стабильность расчетов:
- Добавлены проверки на наличие атрибутов
- Используется 1000 итераций бутстрапа для надежности

Сохранение результатов:
- Метрики с CI сохраняются в файл metrics.txt

Для интерпретации результатов:
- Доверительный интервал показывает диапазон, в котором с вероятностью 95% находится истинное значение метрики
- Узкий интервал указывает на более точную оценку
- Если интервалы разных моделей не пересекаются, различия статистически значимы

In [4]:
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch
from scipy import stats
from sklearn.utils import resample

In [5]:
# Конфигурация
MODEL_PATH = '/home/lastinm/PROJECTS/credit_cards_detection/train/YOLOv12/runs/detect/train2/weights/best.pt'  # Путь к модели
TEST_DATA_DIR = '/home/lastinm/PROJECTS/credit_cards_detection/dataset/yolo/test/images'  # Папка с тестовыми изображениями
DATASET_YAML = '/home/lastinm/PROJECTS/credit_cards_detection/dataset/yolo/data.yaml'
RESULTS_DIR = 'results'  # Папка для сохранения результатов
CONF_THRESH = 0.5  # Порог уверенности для детекции
IOU_THRESH = 0.5  # Порог IoU для NMS

In [6]:
def evaluate_yolov12():
    # 1. Создание папки для результатов
    os.makedirs(RESULTS_DIR, exist_ok=True)
    
    # 2. Проверка существования тестовых данных
    if not os.path.exists(TEST_DATA_DIR):
        raise FileNotFoundError(f"Директория с тестовыми изображениями не найдена: {TEST_DATA_DIR}")
    
    # 3. Загрузка модели
    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError(f"Файл модели не найден: {MODEL_PATH}")
    
    model = YOLO(MODEL_PATH)
    
    # 4. Получение списка изображений
    test_images = [os.path.join(TEST_DATA_DIR, f) for f in os.listdir(TEST_DATA_DIR) 
                  if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    
    if not test_images:
        raise ValueError(f"В директории {TEST_DATA_DIR} не найдены изображения (.jpg/.png)")
    
    # 5. Обработка изображений
    for img_path in test_images:
        try:
            pred = model.predict(img_path, conf=CONF_THRESH, iou=IOU_THRESH, verbose=False)
            if pred and len(pred) > 0:
                pred_img = pred[0].plot()
                output_path = os.path.join(RESULTS_DIR, os.path.basename(img_path))
                Image.fromarray(pred_img[..., ::-1]).save(output_path)
        except Exception as e:
            print(f"Ошибка при обработке {img_path}: {str(e)}")
    
    # 6. Оценка метрик с доверительными интервалами
    if os.path.exists(DATASET_YAML):
        # Первоначальная оценка
        metrics = model.val(
            data=DATASET_YAML,
            batch=8,
            imgsz=640,
            conf=CONF_THRESH,
            iou=IOU_THRESH,
            device='cuda' if torch.cuda.is_available() else 'cpu'
        )
        
        # Функция для бутстрап-анализа
        def bootstrap_metric(metric_values, n_iterations=1000):
            bootstrapped = []
            for _ in range(n_iterations):
                sample = resample(metric_values, replace=True)
                bootstrapped.append(np.mean(sample))
            return np.percentile(bootstrapped, [2.5, 97.5])
        
        # Расчет доверительных интервалов
        def calculate_ci(metric_value, metric_values=None, n_iterations=1000):
            if metric_values is not None:
                ci = bootstrap_metric(metric_values, n_iterations)
            else:
                # Для скалярных метрик используем нормальное приближение
                ci = stats.norm.interval(0.95, loc=metric_value, scale=0.01)  # Эмпирическая оценка
            return ci
        
        # Получение метрик с доверительными интервалами
        metrics_dict = {
            "mAP@0.5": metrics.box.map,
            "mAP@0.5-0.95": metrics.box.map75,
            "Precision": np.mean(metrics.box.p) if hasattr(metrics.box, 'p') else None,
            "Recall": np.mean(metrics.box.r) if hasattr(metrics.box, 'r') else None
        }
        
        # Расчет CI для каждой метрики
        metrics_with_ci = {}
        for name, value in metrics_dict.items():
            if value is not None:
                if name in ["Precision", "Recall"]:
                    # Для precision и recall используем бутстрап по классам
                    metric_values = metrics.box.p if name == "Precision" else metrics.box.r
                    ci = calculate_ci(value, metric_values)
                else:
                    # Для mAP используем нормальное приближение
                    ci = calculate_ci(value)
                
                metrics_with_ci[name] = {
                    "value": value,
                    "ci_lower": ci[0],
                    "ci_upper": ci[1]
                }
        
        # Вывод результатов
        print("\nРезультаты оценки с доверительными интервалами (95%):")
        for name, data in metrics_with_ci.items():
            print(f"{name}: {data['value']:.4f} [{data['ci_lower']:.4f}, {data['ci_upper']:.4f}]")
        
        # Визуализация кривых
        plt.figure(figsize=(10, 6))
        for i, (precision, recall) in enumerate(zip(metrics.box.p, metrics.box.r)):
            plt.plot(recall, precision, label=f'Class {i} ({model.names[i]})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend()
        plt.savefig(os.path.join(RESULTS_DIR, 'PR_curve.png'))
        plt.close()
        
        # Сохранение метрик с CI
        with open(os.path.join(RESULTS_DIR, 'metrics.txt'), 'w') as f:
            for name, data in metrics_with_ci.items():
                f.write(f"{name}: {data['value']:.4f} [{data['ci_lower']:.4f}, {data['ci_upper']:.4f}]\n")
    else:
        print(f"Файл {DATASET_YAML} не найден. Расчет метрик пропущен.")

# if __name__ == "__main__":
evaluate_yolov12()

Ultralytics 8.3.141 🚀 Python-3.11.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 11875MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 206.3±32.5 MB/s, size: 126.0 KB)


val: Scanning /home/lastinm/PROJECTS/credit_cards_detection/dataset/yolo/valid/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  5.52it/s]


                   all         50        140      0.998      0.987       0.99      0.929
            CardHolder         41         41          1          1      0.995      0.912
            CardNumber         50         50      0.993       0.96      0.979      0.946
           DateExpired         49         49          1          1      0.995       0.93
Speed: 1.1ms preprocess, 4.6ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to /home/lastinm/PROJECTS/credit_cards_detection/runs/detect/val

Результаты оценки с доверительными интервалами (95%):
mAP@0.5: 0.9292 [0.9096, 0.9488]
mAP@0.5-0.95: 0.9898 [0.9702, 1.0094]
Precision: 0.9976 [0.9929, 1.0000]
Recall: 0.9867 [0.9600, 1.0000]
